In [2]:
#@title CODE CELL 1: Install & import (safe to re-run)
import sys, subprocess

def pip_install(pkg):
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg], check=False)

# Install required libraries
pip_install("pandas>=1.5")
pip_install("numpy>=1.23")
pip_install("matplotlib>=3.7")
pip_install("scikit-learn>=1.3")

# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [3]:
#@title CODE CELL 2: Create synthetic dataset (our project dataset)

# Random seed for reproducibility
np.random.seed(42)
n_rows = 500

data = pd.DataFrame({
    "Age": np.random.randint(20, 70, n_rows),
    "Height": np.random.randint(150, 200, n_rows),
    "Score": np.random.randint(0, 100, n_rows),
    "Income": np.random.randint(20000, 100000, n_rows),
    "Category": np.random.choice(["A", "B", "C"], n_rows),
    "BMI": np.random.uniform(18, 35, n_rows),
    "Gender": np.random.choice(["Male", "Female"], n_rows),
    "ExerciseHours": np.random.randint(0, 15, n_rows)
})

# Define target variable
data["HealthIndex"] = (
    0.3 * data["Age"] +
    0.2 * data["Height"] -
    0.4 * data["Score"] +
    0.1 * data["Income"]/1000 +
    0.5 * data["BMI"] +
    2.0 * data["ExerciseHours"] +
    np.random.normal(0, 10, n_rows)
)

print("✅ Created synthetic dataset:", data.shape)
data.head()


✅ Created synthetic dataset: (500, 9)


,Age,Height,Score,Income,Category,BMI,Gender,ExerciseHours,HealthIndex
0,58,197,32,51987,B,19.042944,Male,11,83.840868
1,48,170,52,35850,C,31.272947,Female,1,50.320790
2,34,188,21,32112,B,25.816607,Female,7,63.320162
3,62,185,20,44058,A,18.988785,Male,5,95.573968
4,27,182,69,65420,C,34.912727,Male,8,58.624305


In [4]:
#@title CODE CELL 3: Preprocessing
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# 1) Standardize column names
data.columns = data.columns.str.strip().str.lower().str.replace(" ", "_")
print("✅ Standardized column names:", list(data.columns))

# 2) Check duplicates
print("Duplicates count:", data.duplicated().sum())
data = data.drop_duplicates()

# 3) Columns with missing values
print("Missing values per column:")
print(data.isnull().sum())

# 4) Numeric imputation (mean)
num_cols = data.select_dtypes(include=np.number).columns
print("Numeric columns:", list(num_cols))

num_imputer = SimpleImputer(strategy="mean")
data[num_cols] = num_imputer.fit_transform(data[num_cols])

# 5) Categorical imputation (most frequent)
cat_cols = data.select_dtypes(exclude=np.number).columns
print("Categorical columns:", list(cat_cols))

cat_imputer = SimpleImputer(strategy="most_frequent")
data[cat_cols] = cat_imputer.fit_transform(data[cat_cols])

# 6) Encoding strategy (One Hot Encoding)
data = pd.get_dummies(data, drop_first=True)

# 7) Scaling strategy (StandardScaler)
scaler = StandardScaler()
data[num_cols] = scaler.fit_transform(data[num_cols])

# 8) Outlier handling (IQR method)
for col in num_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    outliers = data[(data[col] < lower) | (data[col] > upper)]
    print(f"Outliers in {col}: {len(outliers)}")

# 9) Feature correlation with target
corr = data.corr()
print("\nCorrelation with HealthIndex:")
print(corr["healthindex"].sort_values(ascending=False))



✅ Standardized column names: ['age', 'height', 'score', 'income', 'category', 'bmi', 'gender', 'exercisehours', 'healthindex']
Duplicates count: 0
Missing values per column:
age              0
height           0
score            0
income           0
category         0
bmi              0
gender           0
exercisehours    0
healthindex      0
dtype: int64
Numeric columns: ['age', 'height', 'score', 'income', 'bmi', 'exercisehours', 'healthindex']
Categorical columns: ['category', 'gender']
Outliers in age: 0
Outliers in height: 0
Outliers in score: 0
Outliers in income: 0
Outliers in bmi: 0
Outliers in exercisehours: 0
Outliers in healthindex: 0

Correlation with HealthIndex:
healthindex      1.000000
exercisehours    0.479804
age              0.267623
height           0.158481
bmi              0.119726
income           0.079125
category_C       0.011304
category_B       0.009572
gender_Male      0.003589
score           -0.610309
Name: healthindex, dtype: float64


In [5]:
#@title CODE CELL 4: Correlation with HealthIndex
corr = data.corr(numeric_only=True)
corr["healthindex"].sort_values(ascending=False)


,healthindex
healthindex,1.000000
exercisehours,0.479804
age,0.267623
height,0.158481
bmi,0.119726
income,0.079125
category_C,0.011304
category_B,0.009572
gender_Male,0.003589
score,-0.610309


In [6]:
#@title CODE CELL 5: Train-Test Split & Linear Regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Features and target
X = data.drop(columns=['healthindex'])
y = data['healthindex']

# Ensure categorical encoding
X = pd.get_dummies(X, drop_first=True)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2_test = r2_score(y_test, y_pred)
r2_train = model.score(X_train, y_train)
gap = abs(r2_train - r2_test)

print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("Test R2:", r2_test)
print("Train R2:", r2_train)
print("Train-Test Gap:", gap)


MAE: 0.42955574895880433
MSE: 0.30672875616193723
RMSE: 0.5538309815836753
Test R2: 0.6936667913501504
Train R2: 0.7094264890995339
Train-Test Gap: 0.01575969774938346


In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

# Add constant
X_vif = sm.add_constant(X)

# Convert bool columns to int for VIF
X_vif = X_vif.astype({col: "int" for col in X_vif.select_dtypes(include="bool").columns})

print(X_vif.dtypes)

# Calculate VIF
vif_data = pd.DataFrame()
vif_data["Feature"] = X_vif.columns
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]

print(vif_data)


const            float64
age              float64
height           float64
score            float64
income           float64
bmi              float64
exercisehours    float64
category_B         int64
category_C         int64
gender_Male        int64
dtype: object
         Feature       VIF
0          const  4.113429
1            age  1.007319
2         height  1.010971
3          score  1.013295
4         income  1.017152
5            bmi  1.014122
6  exercisehours  1.020000
7     category_B  1.356467
8     category_C  1.339944
9    gender_Male  1.008461


In [10]:
import statsmodels.api as sm

# Refit model with statsmodels OLS
X_vif_const = sm.add_constant(X)  # add constant if not already added

# Convert boolean columns to int
X_vif_const = X_vif_const.astype({col: "int" for col in X_vif_const.select_dtypes(include="bool").columns})

ols_model = sm.OLS(y, X_vif_const).fit()

# Residuals
residuals = ols_model.resid
print("✅ Residuals computed")

✅ Residuals computed


In [11]:
from scipy.stats import shapiro

shapiro_test = shapiro(residuals)
shapiro_p = shapiro_test[1]
print("Shapiro p-value:", shapiro_p)


Shapiro p-value: 0.8676270053578643


In [12]:
from statsmodels.stats.stattools import durbin_watson

# Durbin–Watson statistic
dw_stat = durbin_watson(residuals)
print("Durbin-Watson statistic:", dw_stat)


Durbin-Watson statistic: 2.163976337465502


In [13]:
from statsmodels.stats.diagnostic import het_breuschpagan

# Breusch–Pagan test
bp_test = het_breuschpagan(residuals, X_vif_const)  # use features + constant
bp_p = bp_test[3]  # p-value
print("Breusch-Pagan p-value:", bp_p)


Breusch-Pagan p-value: 0.6759401456731853


In [14]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
import numpy as np

alphas = [0.01, 0.1, 1, 10, 100]
best_alpha_ridge = None
best_r2_ridge = -np.inf

for a in alphas:
    ridge = Ridge(alpha=a)
    scores = cross_val_score(ridge, X, y, cv=5, scoring='r2')
    mean_r2 = scores.mean()
    if mean_r2 > best_r2_ridge:
        best_r2_ridge = mean_r2
        best_alpha_ridge = a

print("Best Ridge α:", best_alpha_ridge, "CV R²:", best_r2_ridge)


Best Ridge α: 10 CV R²: 0.6899081621754707


In [16]:
cat_cols = data.select_dtypes(include=['object']).columns
print(cat_cols)

Index([], dtype='object')


In [19]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Target
y = data['score']

# Features
X = data.drop(columns=['score'])

# The data has already been one-hot encoded in CODE CELL 3.
# Removing the redundant one-hot encoding step.

# Ensure numeric type
import numpy as np
X = X.astype(float)

print("✅ Feature matrix shape:", X.shape)

✅ Feature matrix shape: (500, 9)


In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

# X before encoding (numeric only)
X_num = data.select_dtypes(include=['int64','float64'])
y = data['score']

model = LinearRegression()
r2_before = cross_val_score(model, X_num, y, cv=5, scoring='r2').mean()
print("R² before encoding:", r2_before)

R² before encoding: 1.0


In [24]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np

# Features (drop target)
X = data.drop(columns=['score'])
y = data['score']

# The data has already been one-hot encoded in CODE CELL 3.
# Removing the redundant one-hot encoding step.

# Ensure numeric type
X_enc = X.astype(float)

# Cross-validation with linear regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

model = LinearRegression()
r2_after = cross_val_score(model, X_enc, y, cv=5, scoring='r2').mean()
print("R² after encoding:", r2_after)

R² after encoding: 0.5294495762404905


In [25]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, train_test_split

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_enc, y, test_size=0.2, random_state=42
)

# Initialize Lasso
lasso = Lasso(max_iter=10000)

# Hyperparameter grid for alpha
alphas = [0.01, 0.1, 1, 10]

# Grid search with cross-validation
grid = GridSearchCV(lasso, param_grid={'alpha': alphas}, cv=5, scoring='r2')
grid.fit(X_train, y_train)

# Best parameters and performance
best_alpha = grid.best_params_['alpha']
best_r2 = grid.best_score_
test_r2 = grid.score(X_test, y_test)
non_zero = np.sum(grid.best_estimator_.coef_ != 0)

# Print results
print("Best alpha:", best_alpha)
print("CV R²:", best_r2)
print("Test R²:", test_r2)
print("# non-zero coefs:", non_zero)


Best alpha: 0.01
CV R²: 0.5130213136433991
Test R²: 0.4834844258678209
# non-zero coefs: 8


In [26]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV, train_test_split
import numpy as np

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, test_size=0.2, random_state=42)

# Alpha values to try
alpha_grid = [0.01, 0.1, 1, 10]

# ----- Lasso -----
lasso = Lasso(max_iter=10000)
grid_lasso = GridSearchCV(lasso, param_grid={'alpha': alpha_grid}, cv=5, scoring='r2')
grid_lasso.fit(X_train, y_train)

lasso_best_alpha = grid_lasso.best_params_['alpha']
lasso_r2_test = grid_lasso.score(X_test, y_test)
lasso_nonzero = np.sum(grid_lasso.best_estimator_.coef_ != 0)

print("Lasso:")
print("Alpha grid:", alpha_grid)
print("Best alpha:", lasso_best_alpha)
print("Test R²:", lasso_r2_test)
print("# Non-zero coefficients:", lasso_nonzero)

# ----- Ridge -----
ridge = Ridge(max_iter=10000)
grid_ridge = GridSearchCV(ridge, param_grid={'alpha': alpha_grid}, cv=5, scoring='r2')
grid_ridge.fit(X_train, y_train)

ridge_best_alpha = grid_ridge.best_params_['alpha']
ridge_r2_test = grid_ridge.score(X_test, y_test)

print("\nRidge:")
print("Alpha grid:", alpha_grid)
print("Best alpha:", ridge_best_alpha)
print("Test R²:", ridge_r2_test)
print("# Non-zero coefficients: all")


Lasso:
Alpha grid: [0.01, 0.1, 1, 10]
Best alpha: 0.01
Test R²: 0.4834844258678209
# Non-zero coefficients: 8

Ridge:
Alpha grid: [0.01, 0.1, 1, 10]
Best alpha: 10
Test R²: 0.48481760061124723
# Non-zero coefficients: all


In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.metrics import mean_squared_error
import numpy as np

cv_folds = 5
model = LinearRegression()

# ----- R² CV -----
r2_scores = cross_val_score(model, X_enc, y, cv=cv_folds, scoring='r2')
r2_mean = r2_scores.mean()
r2_std = r2_scores.std()

# ----- RMSE CV -----
y_pred = cross_val_predict(model, X_enc, y, cv=cv_folds)
rmse = np.sqrt(mean_squared_error(y, y_pred))

# RMSE per fold
kf = KFold(n_splits=cv_folds, shuffle=True, random_state=42)
rmse_per_fold = []
for train_idx, test_idx in kf.split(X_enc):
    model.fit(X_enc.iloc[train_idx], y.iloc[train_idx])
    y_pred_fold = model.predict(X_enc.iloc[test_idx])
    rmse_fold = np.sqrt(mean_squared_error(y.iloc[test_idx], y_pred_fold))
    rmse_per_fold.append(rmse_fold)

rmse_mean = np.mean(rmse_per_fold)
rmse_std = np.std(rmse_per_fold)

print("R² CV mean:", r2_mean)
print("R² CV std:", r2_std)
print("RMSE CV mean:", rmse_mean)
print("RMSE CV std:", rmse_std)

R² CV mean: 0.5294495762404905
R² CV std: 0.06835796216267762
RMSE CV mean: 0.6879548385083274
RMSE CV std: 0.04842437258417522
